In [5]:
import json, os, sys, random, time, copy
from collections import Counter, defaultdict
import numpy as np
from bs4 import BeautifulSoup
import urllib.parse
import urllib.request
from pprint import pprint
import pickle

In [2]:
USER_AGENT_LIST = [
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
            "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
            "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
            "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
            "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
            "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
            "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
            "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.11 TaoBrowser/2.0 Safari/536.11",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; LBBROWSER)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E; LBBROWSER)",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 LBBROWSER",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; QQBrowser/7.0.3698.400)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SV1; QQDownload 732; .NET4.0C; .NET4.0E; 360SE)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
            "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
            "Mozilla/5.0 (iPad; U; CPU OS 4_2_1 like Mac OS X; zh-cn) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8C148 Safari/6533.18.5",
            "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:2.0b13pre) Gecko/20110307 Firefox/4.0b13pre",
            "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:16.0) Gecko/20100101 Firefox/16.0",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11",
            "Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
        ]


In [3]:
def contentUrl_to_displayUrl(contentUrl):
    if 'commons' in contentUrl:
        prefix = 'https://commons.wikimedia.org/wiki/'
    else:
        prefix = 'https://en.wikipedia.org/wiki/'
    tokens = contentUrl.split('/')
    if 'px-' in tokens[-1]:
        return prefix + 'File:'+tokens[-2]
    else:
        return prefix + 'File:'+tokens[-1]

In [4]:
def displayUrl_to_imgUrl(html):

    soup = BeautifulSoup(html, 'html.parser')
    result = soup.find_all('a',class_= "mw-thumbnail-link", limit=10)
    idx = 0
    default = ""
    for r in result:
        soup_short = BeautifulSoup(str(r), 'html.parser')
        for a in soup_short.find_all('a', href=True):
            href = a['href']
            if idx == 0: 
                default = a['href']
                idx += 1
            if '800px' in href:
                return href, "GOOD"
    if len(default)>0: return default, "GOOD"
    else:
        result = soup.find_all('div',class_= "fullImageLink", limit=1)
        for r in result:
            soup_short = BeautifulSoup(str(r), 'html.parser')
            for im in soup_short.find_all('img'):
                if max(int(im['width']), int(im['height']))>=600 and min(int(im['width']), int(im['height']))>=400:
                    return im['src'], "GOOD"
                return im['src'], "SIZE"
    return "", "FIELD_NONEXIST"

In [5]:
def get_display_page_html(desurl):
    start = time.time()
    try:
        req = urllib.request.Request(desurl, headers = {'User-Agent': random.choice(USER_AGENT_LIST)})
        with urllib.request.urlopen(req) as f:
            html = f.read().decode('utf-8')
        #print("url request time: ", time.time() - start)
        return html
    except KeyboardInterrupt:
        raise
    except:
        return ""

In [8]:
def normalize_imgUrl(url):
    if not 'commons' in url: return "https:"+url
    displayUrl = contentUrl_to_displayUrl(url)
    html = get_display_page_html(displayUrl)
    imgUrl, status = displayUrl_to_imgUrl(html)
    return imgUrl

In [17]:
data = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/upd_txt_data/upd_txt_data_{}.json".format(1000), "r"))
try: data_upd_imgUrl = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/upd_txt_data_upd_imgUrl/upd_txt_data_{}.json".format(1000), "r"))
except: data_upd_imgUrl = {}
for k in range(50):
    if str(k) in data_upd_imgUrl: continue
    data_upd_imgUrl[str(k)] = copy.deepcopy(data[str(k)])
    data_upd_imgUrl[str(k)]['img_negFacts'] = []
    for f in data[str(k)]['img_negFacts']:
        if "maps.wikimedia.org" in f['imgUrl']: continue
        f_upd = copy.deepcopy(f)
        f_upd['imgUrl'] = normalize_imgUrl(f['imgUrl'])
        data_upd_imgUrl[str(k)]['img_negFacts'].append(f_upd)
    if k % 10 == 0:
        json.dump(data_upd_imgUrl, open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/upd_txt_data_upd_imgUrl/upd_txt_data_{}.json".format(1000), "w"), indent=4)
#json.dump(data_upd_imgUrl, open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/upd_txt_data_upd_imgUrl/upd_txt_data_{}.json".format(1000), "w"), indent=4)

21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [25]:
path = "/home/yingshac/CYS/WebQnA/WebQnA_data_new/upd_txt_data_16k/"

snippets_count = []
img_count = []

no_snippet_k = []
no_img_k = []

files = os.listdir(path)
data = {}
for f in files:
    if not '.json' in f: continue
    x = json.load(open(os.path.join(path, f), "r"))
    for k in x:
        num_snippets = len(x[k]['new_negFacts'])
        num_imgs = len(x[k]['img_negFacts'])
        if num_snippets == 0:
            no_snippet_k.append(k)
        if num_imgs == 0:
            no_img_k.append(k)
        snippets_count.append(num_snippets)
        img_count.append(num_imgs)
    data.update(x)
print("{} samples finished".format(len(data)))
print(sorted(Counter(snippets_count).items()))
print(sorted(Counter(img_count).items()))
print(len([k for k in data if len(data[k]['new_negFacts'])< 5 or len(data[k]['img_negFacts']) < 5]))

17430 samples finished
[(0, 813), (1, 579), (2, 504), (3, 456), (4, 394), (5, 362), (6, 366), (7, 370), (8, 319), (9, 368), (10, 303), (11, 277), (12, 287), (13, 262), (14, 255), (15, 236), (16, 229), (17, 218), (18, 253), (19, 228), (20, 213), (21, 208), (22, 177), (23, 190), (24, 191), (25, 171), (26, 162), (27, 155), (28, 139), (29, 139), (30, 159), (31, 159), (32, 147), (33, 159), (34, 150), (35, 128), (36, 142), (37, 134), (38, 124), (39, 106), (40, 7198)]
[(0, 330), (1, 696), (2, 636), (3, 615), (4, 537), (5, 515), (6, 459), (7, 454), (8, 430), (9, 443), (10, 394), (11, 364), (12, 331), (13, 330), (14, 339), (15, 305), (16, 292), (17, 281), (18, 305), (19, 260), (20, 251), (21, 219), (22, 217), (23, 223), (24, 195), (25, 224), (26, 199), (27, 220), (28, 144), (29, 190), (30, 186), (31, 159), (32, 159), (33, 158), (34, 148), (35, 152), (36, 146), (37, 125), (38, 147), (39, 139), (40, 5513)]
4213


In [26]:
# 0725: imgUrl was updated so that we'll use the 800px version if it exists. And imgs from map.wikimedia.org were deleted
# 0803: distractor mining done for 16k new txt data 
print(len(data))
#json.dump(data, open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/txt_dataset_0725.json", "w"), indent=4)
json.dump(data, open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/txt_dataset_0803_16k.json", "w"), indent=4)

17430


In [27]:
Counter([data[k]['Split'] for k in data])

Counter({'ind_test': 1805, 'ood_test': 1500, 'train': 12314, 'val': 1811})

In [9]:
src2idx = pickle.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data/dis_src2idx-Copy1.pkl", "rb"))
print(len(src2idx))

340194


In [10]:
for s in src2idx:
    print(type(src2idx[s]))
    print(s)
    print(src2idx[s])
    break

<class 'int'>
https://upload.wikimedia.org/wikipedia/commons/thumb/5/52/Central-Park-18.jpg/800px-Central-Park-18.jpg
10000000


In [28]:
# Assign img ids for neg_img for txt data (starting from 20000000)
#txt_dataset = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/txt_dataset_0725_random_split.json", "r"))

# Assign img ids for neg_img for 16k txt data (starting from 20098051)
txt_dataset = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/txt_dataset_0803_16k.json", "r"))
print(len(txt_dataset))

17430


In [30]:
x_dis_src2idx = pickle.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/x_dis_src2idx.pkl", "rb"))
print(max(x_dis_src2idx.values()))

20098050


In [31]:
#x_dis_src2idx = {}
#idx = 20000000
x_dis_src2idx = pickle.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/x_dis_src2idx.pkl", "rb"))
print(max(x_dis_src2idx.values()))
idx = max(x_dis_src2idx.values()) + 1
for k in txt_dataset:
    for im in txt_dataset[k]['img_negFacts']:
        s = im['imgUrl']
        if s in x_dis_src2idx: continue
        x_dis_src2idx[s] = idx
        idx += 1
print(idx)
print(len(x_dis_src2idx))

20098050
20240700
240700


In [32]:
x_dis_idx2src = {}
for s in x_dis_src2idx:
    idx = x_dis_src2idx[s]
    x_dis_idx2src[idx] = s
print(len(x_dis_idx2src))

240700


In [33]:
# Jul 27 src2idx fixed for negImgs in 7921 txt data
# Aug 3 src2idx fixed for negImgs for 7921 + 16k data
#pickle.dump(x_dis_src2idx, open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/x_dis_src2idx.pkl", "wb"))

In [34]:
# Jul 27 idx2src fixed for negImgs in 7921 txt data
# Aug 3 idx2src fixed for negImgs for 7921 + 16k data
#pickle.dump(x_dis_idx2src, open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/x_dis_idx2src.pkl", "wb"))

In [140]:
dis_idx2src = pickle.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data/dis_idx2src-Copy1.pkl", "rb"))
print(len(dis_idx2src))

340194


In [143]:
dis_idx2src[10002577]

'http://fc02.deviantart.net/fs70/i/2010/216/6/2/STC_Hero_Of_The_Nameless_Zone_by_Saftkalas.jpg'

In [145]:
img_dataset = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/img_dataset_0725.json", "r"))

In [147]:
for k in img_dataset:
    if any([int(g.split('.')[0]) in missing_dis for g in img_dataset[k]['DistractorIds']]):
        print(k, img_dataset[k]['DistractorIds'])

In [136]:
finished_neg = os.listdir("/data/yingshac/MMMHQA/distractors")
finished_neg = [int(k.split('.')[0]) for k in finished_neg]
print([k for k in range(10000000, 10340194) if not k in finished_neg])

[10001050, 10001063, 10001555, 10001593, 10001643, 10001658, 10001939, 10001943, 10002179, 10002185, 10002577, 10003321, 10003665, 10003955, 10005074, 10005410, 10005601, 10005608, 10005617, 10005806, 10006123, 10006638, 10006829, 10006841, 10006843, 10006899, 10006900, 10006901, 10006902, 10006914, 10006916, 10007917, 10008862, 10008958, 10009223, 10009329, 10009331, 10009337, 10010053, 10010377, 10010403, 10010615, 10010787, 10011092, 10011135, 10011915, 10012174, 10012484, 10012493, 10012503, 10012510, 10012511, 10012512, 10012521, 10012715, 10012737, 10012868, 10012891, 10012895, 10013126, 10013222, 10013788, 10014132, 10014143, 10014618, 10014719, 10014822, 10014825, 10014826, 10014827, 10014926, 10015189, 10015190, 10015191, 10015195, 10015511, 10015512, 10015516, 10015573, 10015831, 10016001, 10016021, 10016391, 10016393, 10016394, 10016401, 10016695, 10016781, 10016957, 10016967, 10017590, 10018500, 10018501, 10018502, 10018503, 10018507, 10018694, 10018830, 10019120, 10019124,

In [144]:
missing_dis = [10001050, 10001063, 10001555, 10001593, 10001643, 10001658, 10001939, 10001943, 10002179, 10002185, 10002577, 10003321, 10003665, 10003955, 10005074, 10005410, 10005601, 10005608, 10005617, 10005806, 10006123, 10006638, 10006829, 10006841, 10006843, 10006899, 10006900, 10006901, 10006902, 10006914, 10006916, 10007917, 10008862, 10008958, 10009223, 10009329, 10009331, 10009337, 10010053, 10010377, 10010403, 10010615, 10010787, 10011092, 10011135, 10011915, 10012174, 10012484, 10012493, 10012503, 10012510, 10012511, 10012512, 10012521, 10012715, 10012737, 10012868, 10012891, 10012895, 10013126, 10013222, 10013788, 10014132, 10014143, 10014618, 10014719, 10014822, 10014825, 10014826, 10014827, 10014926, 10015189, 10015190, 10015191, 10015195, 10015511, 10015512, 10015516, 10015573, 10015831, 10016001, 10016021, 10016391, 10016393, 10016394, 10016401, 10016695, 10016781, 10016957, 10016967, 10017590, 10018500, 10018501, 10018502, 10018503, 10018507, 10018694, 10018830, 10019120, 10019124, 10019653, 10019708, 10019773, 10019774, 10019775, 10019780, 10019782, 10019785, 10019870, 10019953, 10019960, 10019988, 10019989, 10020188, 10020299, 10020307, 10020308, 10020310, 10020763, 10020897, 10021260, 10021263, 10021268, 10021271, 10021275, 10021278, 10021350, 10021628, 10021715, 10022022, 10022218, 10022370, 10022479, 10022487, 10022494, 10022577, 10022581, 10023254, 10023255, 10023960, 10023964, 10023969, 10023995, 10024372, 10024812, 10024821, 10025161, 10025246, 10025536, 10025541, 10025548, 10025895, 10026701, 10026702, 10026705, 10026888, 10026891, 10027957, 10028078, 10028349, 10028392, 10028656, 10028673, 10028770, 10028823, 10028824, 10029037, 10029065, 10029262, 10029670, 10029672, 10029774, 10029804, 10029806, 10029825, 10030050, 10031050, 10031090, 10031386, 10031387, 10031391, 10031403, 10031957, 10031959, 10031968, 10032112, 10032529, 10032571, 10032939, 10032941, 10032944, 10033470, 10033483, 10033808, 10033830, 10033838, 10034471, 10034485, 10034486, 10034532, 10034542, 10034761, 10034923, 10035150, 10035151, 10035155, 10035156, 10035157, 10035163, 10035164, 10035344, 10035348, 10035349, 10035416, 10035419, 10035420, 10035458, 10035493, 10035841, 10036033, 10036399, 10036576, 10036578, 10037171, 10037172, 10037323, 10037343, 10037352, 10037469, 10037663, 10037825, 10037829, 10037830, 10037835, 10038456, 10038459, 10038571, 10038576, 10038577, 10038578, 10038580, 10038581, 10038583, 10038647, 10039164, 10039169, 10039170, 10039171, 10039546, 10040351, 10040373, 10040419, 10040557, 10040656, 10040777, 10040930, 10040942, 10040953, 10041308, 10041563, 10041769, 10042059, 10043361, 10043449, 10043461, 10043795, 10043796, 10044033, 10044035, 10044036, 10044125, 10044616, 10044955, 10044997, 10045628, 10045706, 10045786, 10045796, 10046050, 10046360, 10046543, 10046681, 10046874, 10047263, 10047331, 10047333, 10047334, 10047338, 10047348, 10047350, 10047392, 10047794, 10047867, 10048279, 10048306, 10048722, 10049236, 10049554, 10049555, 10049556, 10049559, 10049800, 10049863, 10049951, 10050523, 10051149, 10051314, 10051963, 10052524, 10052895, 10053194, 10053201, 10053205, 10053448, 10053570, 10053810, 10053981, 10053984, 10053985, 10054934, 10054977, 10055113, 10055369, 10055634, 10056508, 10056710, 10056749, 10056916, 10056919, 10057230, 10057898, 10058165, 10058472, 10058723, 10058930, 10060088, 10060103, 10060116, 10060337, 10060505, 10061458, 10061460, 10061754, 10061824, 10061834, 10061959, 10062884, 10063038, 10063627, 10063628, 10063629, 10063923, 10063934, 10064566, 10064892, 10065090, 10065695, 10065808, 10065992, 10065997, 10066000, 10066010, 10066019, 10066229, 10066230, 10066635, 10066865, 10067007, 10068037, 10068819, 10069513, 10070051, 10070904, 10071404, 10071406, 10071557, 10071659, 10071773, 10072008, 10072087, 10072421, 10072857, 10072861, 10073185, 10073606, 10074837, 10075246, 10075253, 10075659, 10076056, 10076666, 10077443, 10077917, 10078008, 10078341, 10078576, 10078760, 10080952, 10082220, 10082253, 10082482, 10083652, 10083655, 10083656, 10083661, 10084206, 10084284, 10084292, 10084296, 10084297, 10084352, 10084695, 10085050, 10085122, 10085125, 10085379, 10085888, 10086013, 10086110, 10086126, 10086713, 10087381, 10087525, 10087846, 10087985, 10088125, 10088127, 10088752, 10089310, 10089312, 10089316, 10089534, 10089698, 10089945, 10090216, 10090351, 10090660, 10091068, 10091429, 10091899, 10092191, 10092236, 10092762, 10093696, 10093789, 10093837, 10094062, 10094151, 10094257, 10094531, 10094976, 10095133, 10095143, 10095498, 10095560, 10096135, 10096544, 10097767, 10097783, 10097863, 10097960, 10098142, 10098182, 10098464, 10098540, 10098916, 10099303, 10099500, 10099831, 10099945, 10100532, 10100638, 10100640, 10100641, 10101228, 10101475, 10101477, 10101478, 10102030, 10102032, 10102041, 10102049, 10102081, 10102235, 10102769, 10102901, 10103608, 10103609, 10103612, 10103790, 10104368, 10105053, 10105111, 10105113, 10105115, 10105388, 10105935, 10106285, 10106927, 10107176, 10107178, 10107181, 10107915, 10108075, 10108323, 10108330, 10108334, 10109432, 10109668, 10110001, 10110049, 10110050, 10110059, 10111054, 10111064, 10112460, 10112631, 10113413, 10114704, 10114891, 10115703, 10116598, 10117170, 10118025, 10118510, 10118685, 10118748, 10118867, 10119068, 10119319, 10119423, 10119466, 10119859, 10119891, 10120071, 10120290, 10120471, 10120474, 10120501, 10120504, 10120512, 10120806, 10121101, 10121187, 10121601, 10121604, 10121612, 10121616, 10121893, 10122072, 10122073, 10122107, 10122121, 10122163, 10122175, 10122227, 10122233, 10122972, 10123007, 10123772, 10123782, 10125038, 10125152, 10125193, 10125225, 10125228, 10125230, 10125603, 10125684, 10125685, 10125985, 10126664, 10126705, 10126711, 10127521, 10127560, 10127613, 10127708, 10128137, 10128150, 10128878, 10129128, 10129129, 10129302, 10129750, 10130288, 10130874, 10130878, 10131175, 10131659, 10131708, 10132601, 10132616, 10132913, 10133256, 10133318, 10133328, 10133406, 10133409, 10133410, 10133793, 10134161, 10134261, 10134267, 10134268, 10135026, 10135247, 10135298, 10135367, 10135869, 10136060, 10136408, 10136512, 10137126, 10137641, 10138232, 10138502, 10138508, 10138510, 10138512, 10139752, 10139760, 10139764, 10139846, 10139847, 10139852, 10139953, 10140273, 10140275, 10140277, 10140538, 10141029, 10141080, 10141726, 10141727, 10142007, 10142189, 10142812, 10143873, 10143878, 10144738, 10145158, 10146202, 10146268, 10146270, 10146272, 10146273, 10146647, 10146649, 10146650, 10147445, 10147450, 10147451, 10147697, 10147995, 10148360, 10148513, 10148726, 10148795, 10148803, 10148807, 10149072, 10149075, 10149561, 10150114, 10150171, 10150173, 10150178, 10150182, 10150506, 10150947, 10151115, 10151143, 10151146, 10152009, 10152280, 10152358, 10152362, 10152367, 10152368, 10152993, 10153687, 10154062, 10154863, 10155083, 10155084, 10155406, 10156060, 10156386, 10157661, 10157691, 10157721, 10157837, 10157839, 10157920, 10157948, 10158589, 10158605, 10158809, 10158812, 10158815, 10158816, 10158867, 10158868, 10158869, 10158870, 10159004, 10159342, 10159583, 10159585, 10159592, 10159609, 10160450, 10160864, 10161430, 10161548, 10161645, 10162882, 10163694, 10163704, 10163838, 10164170, 10164189, 10164211, 10164348, 10164425, 10164878, 10164897, 10164974, 10165110, 10165381, 10165437, 10165440, 10165450, 10165451, 10165847, 10166704, 10166866, 10166913, 10167090, 10168249, 10169065, 10169171, 10169583, 10170070, 10170772, 10170800, 10170801, 10170804, 10171077, 10171266, 10171398, 10172040, 10172811, 10173183, 10173387, 10173394, 10173396, 10173399, 10173400, 10173889, 10174121, 10174218, 10174489, 10174873, 10174876, 10175473, 10175476, 10175479, 10175490, 10175573, 10177170, 10177458, 10177599, 10177693, 10177889, 10177941, 10178192, 10180208, 10180721, 10180862, 10181518, 10181881, 10181952, 10182227, 10182936, 10182939, 10182952, 10183170, 10183528, 10184127, 10184634, 10184926, 10185701, 10185884, 10186433, 10186910, 10187591, 10187690, 10188048, 10188063, 10188327, 10188328, 10188329, 10188330, 10188674, 10188678, 10188680, 10189556, 10189617, 10189625, 10189631, 10189632, 10189805, 10189807, 10190153, 10190162, 10190334, 10190524, 10190716, 10190860, 10190876, 10190887, 10190927, 10191031, 10191035, 10191038, 10191096, 10192158, 10192243, 10192714, 10192815, 10193270, 10193640, 10193969, 10194177, 10194207, 10194648, 10194949, 10195671, 10196417, 10197322, 10197324, 10197328, 10197738, 10197740, 10197808, 10198575, 10198723, 10198978, 10199429, 10199988, 10200076, 10200078, 10200081, 10200953, 10200986, 10201474, 10202409, 10202541, 10202670, 10202751, 10202758, 10203139, 10203140, 10203154, 10203213, 10203392, 10203536, 10203579, 10203971, 10204244, 10204245, 10204247, 10204250, 10204252, 10204254, 10204365, 10204510, 10204528, 10204686, 10204687, 10204691, 10204692, 10204697, 10204825, 10205528, 10205550, 10206006, 10206593, 10207907, 10208355, 10208552, 10208554, 10208560, 10208821, 10209277, 10210631, 10211511, 10211564, 10211865, 10211869, 10211875, 10211878, 10212678, 10212781, 10213104, 10213565, 10213644, 10214234, 10214684, 10214828, 10214972, 10215320, 10215608, 10216246, 10216459, 10216849, 10217171, 10217206, 10217402, 10217572, 10218196, 10218298, 10218308, 10218568, 10218786, 10219862, 10220054, 10220057, 10220063, 10220073, 10220074, 10220075, 10220079, 10220080, 10220169, 10220331, 10220632, 10220639, 10221048, 10222169, 10222367, 10222688, 10223407, 10224021, 10224025, 10224442, 10224791, 10224794, 10224860, 10224884, 10225236, 10226815, 10227245, 10227357, 10227459, 10228463, 10228465, 10228472, 10228473, 10228474, 10228954, 10229340, 10229810, 10230093, 10230098, 10230101, 10230963, 10231096, 10231098, 10231108, 10231114, 10231775, 10231777, 10232199, 10233254, 10233864, 10234009, 10234107, 10234222, 10234243, 10234425, 10235100, 10235496, 10235505, 10235838, 10236209, 10236451, 10236838, 10236863, 10237966, 10238151, 10238317, 10238318, 10239343, 10239855, 10239858, 10240025, 10241046, 10241047, 10241049, 10242432, 10242433, 10242434, 10242436, 10242439, 10243578, 10243805, 10244256, 10244568, 10245693, 10245893, 10245910, 10245911, 10246081, 10246483, 10246696, 10248768, 10248899, 10248906, 10248908, 10249492, 10249535, 10249727, 10249766, 10250059, 10250348, 10250773, 10251260, 10251261, 10251288, 10252092, 10252217, 10252634, 10252681, 10252689, 10252944, 10253469, 10253471, 10254395, 10254461, 10255042, 10255091, 10255097, 10255100, 10255420, 10255517, 10256638, 10256800, 10256839, 10257559, 10257641, 10257831, 10258424, 10258461, 10259173, 10259396, 10259453, 10259839, 10259848, 10260752, 10261250, 10261773, 10261893, 10262012, 10262251, 10262405, 10262540, 10262726, 10262732, 10262734, 10263882, 10264398, 10264564, 10265078, 10265298, 10265631, 10265650, 10265652, 10266226, 10266427, 10266429, 10266452, 10266501, 10267267, 10267273, 10267276, 10267726, 10267963, 10268181, 10268706, 10269335, 10269464, 10269466, 10269470, 10269907, 10270509, 10271533, 10271876, 10272385, 10272797, 10274062, 10274120, 10274613, 10274763, 10274879, 10274923, 10275393, 10275445, 10275634, 10276076, 10276239, 10276240, 10276241, 10277044, 10277091, 10277093, 10277097, 10277100, 10277301, 10277721, 10277728, 10277731, 10278070, 10278397, 10278920, 10279075, 10279272, 10279618, 10279625, 10279627, 10279628, 10279726, 10282397, 10282400, 10282401, 10282404, 10282532, 10283064, 10283386, 10284899, 10284901, 10284902, 10284903, 10284905, 10284907, 10285205, 10286306, 10286598, 10286872, 10287316, 10287800, 10287801, 10287805, 10288085, 10288119, 10288411, 10288458, 10288932, 10289741, 10289859, 10290963, 10290968, 10290971, 10290972, 10291330, 10291351, 10291558, 10292209, 10292293, 10292585, 10292751, 10293101, 10293283, 10293286, 10293549, 10293991, 10294111, 10294194, 10294202, 10294879, 10295294, 10295470, 10295473, 10295650, 10295966, 10295979, 10296184, 10296298, 10296574, 10296718, 10296735, 10296738, 10298366, 10298909, 10299153, 10299661, 10299739, 10300709, 10301262, 10301427, 10301460, 10301673, 10301675, 10301680, 10301916, 10301921, 10302317, 10304784, 10305081, 10305368, 10305449, 10305804, 10305805, 10305806, 10306417, 10306654, 10306655, 10306666, 10306818, 10306887, 10306889, 10307045, 10307213, 10307453, 10307477, 10307668, 10307701, 10307707, 10308261, 10308534, 10308569, 10309202, 10309257, 10309488, 10309514, 10309816, 10309846, 10310569, 10310934, 10311581, 10311755, 10312619, 10312889, 10312891, 10312893, 10313078, 10313083, 10313085, 10313086, 10313097, 10313162, 10313163, 10313164, 10313165, 10313242, 10313359, 10313719, 10313860, 10313864, 10314272, 10314326, 10314327, 10314328, 10314330, 10314723, 10314742, 10314743, 10315483, 10316627, 10317403, 10317493, 10317753, 10317889, 10318177, 10318178, 10318647, 10319348, 10319405, 10319499, 10319529, 10319700, 10319947, 10320610, 10321903, 10321904, 10321907, 10321913, 10322198, 10322227, 10322349, 10323082, 10323292, 10323782, 10323962, 10324163, 10324841, 10325118, 10325125, 10325158, 10326026, 10326291, 10326477, 10326802, 10327024, 10327128, 10327255, 10327387, 10327394, 10327910, 10327911, 10327913, 10327914, 10328133, 10329373, 10330488, 10330855, 10331444, 10331866, 10332657, 10332659, 10332660, 10332664, 10333511, 10333841, 10334370, 10334464, 10334467, 10335073, 10335213, 10335450, 10335451, 10335452, 10335990, 10336169, 10336173, 10336395, 10336424, 10336554, 10336966, 10337071, 10337250, 10338120, 10338309, 10338379, 10338501, 10338531, 10339548, 10339650, 10340012, 10340097]

In [119]:
finished_x_neg = os.listdir("/data/yingshac/MMMHQA/x_distractors")
finished_x_neg = [int(k.split('.')[0]) for k in finished_x_neg]

In [129]:
[k for k in range(20000000, 20240700) if not k in finished_x_neg]

[20002804,
 20012291,
 20014583,
 20020053,
 20025765,
 20027340,
 20032059,
 20033356,
 20036503,
 20037252,
 20045857,
 20053166,
 20080677,
 20083075,
 20083080,
 20084638,
 20090783,
 20094021,
 20098527,
 20108105,
 20108112,
 20108121,
 20115189,
 20119191,
 20133490,
 20135292,
 20152099,
 20157903,
 20169848,
 20180266,
 20183784,
 20185427,
 20192186,
 20192643,
 20221085,
 20224311,
 20231197,
 20236330,
 20238819]

In [127]:
import yaml
linelist_files = os.listdir("/home/yingshac/CYS/WebQnA/VinVL/tmp/22265_linelist_chunks")
for i in range(23):
    linelist_file = '{}_{}_imgs.test.linelist.tsv'.format(i*100, i*100+99)
    assert linelist_file in linelist_files
    d = {'img': 'WebQnA_img.tsv',
         'hw': 'WebQnA_hw.tsv', 
         'label': 'label_danfeiX_clipped.new.tsv', 
         'labelmap': 'VG-SGG-dicts-danfeiX-clipped.json', 
         'linelist': linelist_file}
    with open('/home/yingshac/CYS/WebQnA/VinVL/tmp/22265_yaml_chunks/WebQnA_{}_{}_imgs.yaml'.format(i*100, i*100+99), 'w') as yaml_file:
        yaml.dump(d, yaml_file, default_flow_style=False)

In [85]:
import requests
import urllib.request
import glob
import time
import random
from datetime import datetime
from pytz import timezone

In [102]:
idx2src = pickle.load(open('/home/yingshac/CYS/WebQnA/WebQnA_data_new/x_dis_idx2src.pkl', 'rb'))

def download(start, end, folder_path):
    assert end > start
    head  = 10000*(start // 10000 + 1)
    with open("DownloadLog_{}.txt".format(head), 'a') as file:
        file.write(datetime.now(tz=timezone('US/Eastern')).strftime("%y-%m-%d %H:%M:%S") + '\n')
        file.write("head = {}, start = {}, end = {} \n".format(head, start, end))
    existing_files = os.listdir(folder_path)
    for i in list(idx2src.keys())[start:end]:
        #name_str = "0"*(8-len(str(i))) + str(i)
        if str(i)+".jpg" in existing_files: 
            continue
        #USER_AGENT_LIST = ['wikipedia (https://github.com/goldsmith/Wikipedia/)']
        USER_AGENT_LIST = [
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
            "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
            "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
            "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
            "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
            "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
            "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
            "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.11 TaoBrowser/2.0 Safari/536.11",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; LBBROWSER)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E; LBBROWSER)",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 LBBROWSER",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; QQBrowser/7.0.3698.400)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SV1; QQDownload 732; .NET4.0C; .NET4.0E; 360SE)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
            "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
            "Mozilla/5.0 (iPad; U; CPU OS 4_2_1 like Mac OS X; zh-cn) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8C148 Safari/6533.18.5",
            "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:2.0b13pre) Gecko/20110307 Firefox/4.0b13pre",
            "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:16.0) Gecko/20100101 Firefox/16.0",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11",
            "Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
        ]
        try: 
            user_agent = random.choice(USER_AGENT_LIST)
            response = requests.get(idx2src[i], headers = {'User-Agent': user_agent})
            #while response.status_code == 429:
                #time.sleep(3)
                #user_agent = random.choice(USER_AGENT_LIST)
                #response = requests.get(update_imgUrl(idx2src[i]), headers = {'User-Agent': user_agent})
            if not response.status_code == 200:
                response = requests.get(idx2src[i], headers = {'User-Agent': user_agent})
                if response.status_code == 404:
                    with open(problematic_imgs.txt, 'a') as file:
                        file.write("404 !!! dis_id = {}\n{}\n".format(i, idx2src[i]))
                    continue
                while not response.status_code == 200:
                    time.sleep(3)
                    response = requests.get(idx2src[i])
            #print("get ", i)
            file = open(os.path.join(folder_path, "{}.jpg".format(i)), "wb")
            file.write(response.content)
            file.close()
            if (i+1)%100 == 0:
                with open("DownloadLog_{}.txt".format(head), 'a') as file:
                    file.write("Finish [{}, {}]: {}\n".format(start, end, i))
        except KeyboardInterrupt:
            raise
        except:
            with open(problematic_imgs.txt, 'a') as file:
                file.write(datetime.now(tz=timezone('US/Eastern')).strftime("%y-%m-%d %H:%M:%S") + '\n')
                file.write("Error when downloading id={}, {}\n".format(i, idx2src[i]))
    with open("DownloadLog_{}.txt".format(head), 'a') as file:
        file.write("Finish {} !!! \n".format(head))


    
download(135292, 140000, "/data/yingshac/MMMHQA/x_distractors/")

KeyboardInterrupt: 

In [104]:
USER_AGENT_LIST = [
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
            "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
            "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
            "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
            "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
            "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
            "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
            "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.11 TaoBrowser/2.0 Safari/536.11",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; LBBROWSER)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E; LBBROWSER)",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 LBBROWSER",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; QQBrowser/7.0.3698.400)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SV1; QQDownload 732; .NET4.0C; .NET4.0E; 360SE)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
            "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
            "Mozilla/5.0 (iPad; U; CPU OS 4_2_1 like Mac OS X; zh-cn) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8C148 Safari/6533.18.5",
            "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:2.0b13pre) Gecko/20110307 Firefox/4.0b13pre",
            "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:16.0) Gecko/20100101 Firefox/16.0",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11",
            "Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
]
aval_USER_AGENT_LIST = []
for ua in USER_AGENT_LIST:
    response = requests.get(idx2src[20135293], headers = {'User-Agent': ua})
    if response.status_code == 200:
        aval_USER_AGENT_LIST.append(ua)
print(aval_USER_AGENT_LIST)

['Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)', 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)', 'Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)', 'Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)', 'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)', 'Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)', 'Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)', 'Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Sa

In [98]:
print(len(aval_USER_AGENT_LIST), len(USER_AGENT_LIST))

35 35
